In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import utils
from tensorflow.keras import preprocessing

/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
train = pd.read_csv('data/train.csv')
train.head()

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
id       60000 non-null int64
label    60000 non-null int64
dtypes: int64(2)
memory usage: 937.6 KB


In [5]:
# load our grayscale images
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('data/train/'+train['id'][i].astype('str')+'.png', target_size=(28,28,1), color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 60000/60000 [00:35<00:00, 1704.63it/s]


## Data preparation

In [6]:
y=train['label'].values
y = to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## Define our model structure

example from tf.keras website: <br>
`model = tf.keras.Sequential()`<br>
Adds a densely-connected layer with 64 units to the model:<br>
`model.add(layers.Dense(64, activation='relu'))`<br>
Add another:<br>
`model.add(layers.Dense(64, activation='relu'))`<br>
Add a softmax layer with 10 output units:<br>
`model.add(layers.Dense(10, activation='softmax'))`<br>

example from project: <br>
# Define the model structure<br>
model = Sequential()<br>
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))<br>
model.add(Conv2D(64, (3, 3), activation='relu'))<br>
model.add(MaxPooling2D(pool_size=(2, 2)))<br>
model.add(Dropout(0.25))<br>
model.add(Flatten())<br>
model.add(Dense(128, activation='relu'))<br>
model.add(Dropout(0.5))<br>
model.add(Dense(10, activation='softmax'))<br>

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [10]:
# Training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 54s 1ms/step - loss: 0.3911 - acc: 0.8595 - val_loss: 0.2846 - val_acc: 0.8945
Epoch 2/10
48000/48000 [==============================] - 55s 1ms/step - loss: 0.2470 - acc: 0.9101 - val_loss: 0.2469 - val_acc: 0.9086
Epoch 3/10
48000/48000 [==============================] - 54s 1ms/step - loss: 0.1923 - acc: 0.9280 - val_loss: 0.2413 - val_acc: 0.9157
Epoch 4/10
48000/48000 [==============================] - 53s 1ms/step - loss: 0.1477 - acc: 0.9453 - val_loss: 0.2369 - val_acc: 0.9173
Epoch 5/10
48000/48000 [==============================] - 52s 1ms/step - loss: 0.1102 - acc: 0.9597 - val_loss: 0.2511 - val_acc: 0.9203
Epoch 6/10
48000/48000 [==============================] - 52s 1ms/step - loss: 0.0804 - acc: 0.9702 - val_loss: 0.2659 - val_acc: 0.9236
Epoch 7/10
48000/48000 [==============================] - 51s 1ms/step - loss: 0

In [11]:
tf.__version__

'1.13.1'

## 0.9207 Accuracy is pretty good, let's persist it.

In [14]:
import pickle

In [15]:
filename = 'clothes_model0.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
import pickle

In [20]:
import dill

In [21]:
with open('dl_clothes_model.dl', 'wb') as file:
    dill.dump(model, file)